In [1]:
'''
author: Sha Yuan
time: 2018-06-05
funtion: task2 is to predict given scholars' research interest.
'''

import pandas as pd
import numpy as np
import sklearn
import gensim
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
import time
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

'''
# read file and store strings (which is seperated by '\n') into a list
# filter the '' （empty string), which is the empty rows in train.txt
'''
# preprocess train.txt
with open('/Users/yuansha/Desktop/portrait/training.txt', 'r', encoding = 'utf-8') as f:
    train = filter(lambda str: str != '', f.read().split('\n')[:-1])
train = list(train)
print(train[0:10])

Using TensorFlow backend.


['Daniel J. Paulish', 'software engineering,software architecture,software design', 'José Cristóbal Riquelme Santos', 'neural networks,machine learning,evolutionary algorithm', 'Fernando C. N. Pereira', 'computer science,natural language processing,speech recognition', 'Juan de Lara', 'computer science,software engineering,model checking', 'Stefano Lonardi', 'computational biology,bioinformatics,data mining']


In [2]:
authors = []
labels1 = []
labels2 = []
labels3 = []
for idx, item in enumerate(train):
    if idx % 2 == 0:
        authors.append(item)
    else:
        label1, label2, label3 = item.split(',')
        labels1.append(label1)
        labels2.append(label2)
        labels3.append(label3)
dic = {'author': authors, 'Interest1': labels1, 'Interest2': labels2, 'Interest3': labels3}
dfTrain = pd.DataFrame(dic)
print(dfTrain[0:10])


                   Interest1                    Interest2  \
0       software engineering        software architecture   
1            neural networks             machine learning   
2           computer science  natural language processing   
3           computer science         software engineering   
4      computational biology               bioinformatics   
5        virtual environment          knowledge discovery   
6          computer graphics              computer vision   
7          genetic algorithm          dynamic programming   
8  computational photography              virtual reality   
9          genetic algorithm          computer simulation   

                       Interest3                          author  
0                software design               Daniel J. Paulish  
1         evolutionary algorithm  José Cristóbal Riquelme Santos  
2             speech recognition          Fernando C. N. Pereira  
3                 model checking                    Juan de 

In [3]:
'''
# 合并dfTrain里所有相同interest，并统计其出现的次数
interest_count = dfTrain['Interest1'].value_counts()
for i in range(2,4):
    interest_count = interest_count.add(dfTrain['Interest' + str(i)].value_counts(), fill_value = 0)
'''

# preprocess task2_test_final.txt to dfTest, and combine it with dfTrain
test = open('/Users/yuansha/Desktop/portrait/data of task2/task2_test_final.txt', 'r').read().split('\n')[:-1]
dfTest = pd.DataFrame(test)
# dfTest.rename(columns = {0: 'Name'}) # why not work here?
dfTest.columns = ['author']
# without sort = False, the 'Name' column will be put in the last column, because it is longest.
#dfTotal = pd.concat([dfTrain, dfTest], axis = 0, sort = False).fillna('')
dfTotal = pd.concat([dfTrain, dfTest], axis = 0).fillna('')
dfTotal.index = dfTotal['author']
print(dfTotal[0:5])
print(dfTotal[-5:-1])


                                            Interest1  \
author                                                  
Daniel J. Paulish                software engineering   
José Cristóbal Riquelme Santos        neural networks   
Fernando C. N. Pereira               computer science   
Juan de Lara                         computer science   
Stefano Lonardi                 computational biology   

                                                  Interest2  \
author                                                        
Daniel J. Paulish                     software architecture   
José Cristóbal Riquelme Santos             machine learning   
Fernando C. N. Pereira          natural language processing   
Juan de Lara                           software engineering   
Stefano Lonardi                              bioinformatics   

                                             Interest3  \
author                                                   
Daniel J. Paulish                      sof

In [4]:
#读取papers.txt，将其转换成pandas的DataFrame格式
papers = []
with open('/Users/yuansha/Desktop/portrait/data of task3/papers.txt', 'r', encoding = 'utf-8') as f:
    for line in f.readlines():
        if line.startswith('#index'):
            paper_info = {}
            papers.append(paper_info)
            paper_info['index'] = line[6:].strip()
            paper_info['citation'] = ''
        elif line.startswith('#@'):
            paper_info['author'] = line[2:].strip()
        elif line.startswith('#*'):
            paper_info['paper'] = line[2:].strip()
        elif line.startswith('#t'):
            paper_info['year'] = line[2:].strip()
        elif line.startswith('#c'):
            paper_info['journal'] = line[2:].strip()
        elif line.startswith('#%'):
            paper_info['citation'] = paper_info['citation']+line[2:]+','
        elif (line == '\n' or line == ''):
            continue
dfPapers = pd.DataFrame(papers)
dfPapers.index = dfPapers['index']
dfPapers = dfPapers.dropna()
print(dfPapers[0:10])

                                                  author  \
index                                                      
0                          Sinjini Mitra,Marios Savvides   
1      Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2            Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga   
3                                Yilmaz Erdem,Ümit Totur   
4      Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   
5                                                Song Li   
6      Lisa A. Osborne,Rodden Middleton,Kerina H. Jon...   
7                                            Bodo Möller   
8                     Mototsugu Nishioka,Naohisa Komatsu   
9      Kleanthis C. Neokleous,Marios N. Avraamides,An...   

                                                citation index  \
index                                                            
0                                                            0   
1      1775098\n,1648782\n,2206608\n,2857290\n,267443...     1   
2              

In [5]:
'''
处理机构名和文章标题，方便做tfidf
处理方法包括：删除标点符号，统一大小写
'''
def removeSpace(x):
    new_x = ''
    for i in x.split(' '):
        new_x += i
    return new_x
'''
将期刊名+期刊名作为一个单词+论文标题组成一篇文档
例如：期刊名为Computer Vision, 论文标题为convolution neural networks
则生成文档为: Computer Vision convolution neural networks Computer_Vision
'''
#pd.set_option('display.max_rows',500)
#pd.set_option('display.max_columns',500)
#pd.set_option('display.width',1000)

dfPapers['jour_'] = dfPapers['journal'].apply(removeSpace)
dfPapers['jour_paper'] = dfPapers['journal']+' '+dfPapers['paper']+dfPapers['jour_']
jour_list = list(dfPapers['jour_'].unique())
dfPapers.drop('jour_', axis = 1, inplace = True)
print(dfPapers.loc[['0','1','2'],['jour_paper']])
a = list(dfPapers.loc['1',['jour_paper']])
print(a)

                                              jour_paper
index                                                   
0      ICASSP (2) Improved Human Face Identification ...
1      Pattern Recognition Unsupervised methods for t...
2      ISCAS (6) A post layout watermarking method fo...
['Pattern Recognition Unsupervised methods for the classification of hyperspectral images with low spatial resolution.PatternRecognition']


In [6]:
def normalize_text(text):
    norm_text = text.lower()
    norm_text = norm_text.replace('<br />', ' ')
    for char in ['.', '"', ',', '(', ')', '!', '?', ':', ';', '-']:
        norm_text = norm_text.replace(char, ' '+char+' ')
    return norm_text

dfPapers['jour_paper'] = dfPapers['jour_paper'].apply(normalize_text)
docs = list(dfPapers['jour_paper'].values)
print(docs[:10])


['icassp  ( 2 )  improved human face identification using frequency domain representation of facial asymmetry . icassp ( 2 ) ', 'pattern recognition unsupervised methods for the classification of hyperspectral images with low spatial resolution . patternrecognition', 'iscas  ( 6 )  a post layout watermarking method for ip protection . iscas ( 6 ) ', 'computers & mathematics with applications some tauberian theorems for the product method of borel and cesàro summability . computers&mathematicswithapplications', 'ieee t .  geoscience and remote sensing mirs :  an all - weather 1dvar satellite data assimilation and retrieval system . ieeet . geoscienceandremotesensing', 'journal of approximation theory convergence rates of vector cascade algorithms in lp . journalofapproximationtheory', 'i .  j .  medical informatics desirability and expectations of the uk ms register :  views of people with ms . i . j . medicalinformatics', 'icisc fractional windows revisited :  improved signed - digit r

In [7]:

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()
    docs[idx] = tokenizer.tokenize(docs[idx])
print(docs[:50])


[['icassp', '2', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'icassp', '2'], ['pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'of', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition'], ['iscas', '6', 'a', 'post', 'layout', 'watermarking', 'method', 'for', 'ip', 'protection', 'iscas', '6'], ['computers', 'mathematics', 'with', 'applications', 'some', 'tauberian', 'theorems', 'for', 'the', 'product', 'method', 'of', 'borel', 'and', 'cesàro', 'summability', 'computers', 'mathematicswithapplications'], ['ieee', 't', 'geoscience', 'and', 'remote', 'sensing', 'mirs', 'an', 'all', 'weather', '1dvar', 'satellite', 'data', 'assimilation', 'and', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing'], ['journal', 'of', 'approximation', 'theory', 'convergence', 'rates', 'of', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofappr

In [1]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()
    docs[idx] = tokenizer.tokenize(docs[idx])
print(docs[:10])

NameError: name 'docs' is not defined

In [8]:
#Remove numbers
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

#Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

print(docs[:50])

[['icassp', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'icassp'], ['pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'of', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition'], ['iscas', 'post', 'layout', 'watermarking', 'method', 'for', 'ip', 'protection', 'iscas'], ['computers', 'mathematics', 'with', 'applications', 'some', 'tauberian', 'theorems', 'for', 'the', 'product', 'method', 'of', 'borel', 'and', 'cesàro', 'summability', 'computers', 'mathematicswithapplications'], ['ieee', 'geoscience', 'and', 'remote', 'sensing', 'mirs', 'an', 'all', 'weather', '1dvar', 'satellite', 'data', 'assimilation', 'and', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing'], ['journal', 'of', 'approximation', 'theory', 'convergence', 'rates', 'of', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofapproximationtheory'], ['medical',

In [9]:
#get word_list
def processDoc(docs):
    wordList = {}
    for doc in docs:
        for word in doc:
            if word not in wordList.keys():
                wordList[word] = 1
            else:
                wordList[word]+=1
    return wordList
# 删去一些可能无用的此词？
wordList = processDoc(docs)
'''
# 打印word_list字典
ci = 0
for item in wordList.items():
    ci += 1
    if ci <100:
        print(item)
    else:
        break
'''
word_ind = list(wordList.keys())
word_val = list(wordList.values())
wordList = pd.DataFrame(word_val)
wordList.index = word_ind
#print(wordList[:100])
# 取出现次数超过20的词加入到tfidf 的计算中
wordList = list(wordList[wordList[0]>20].index)
print(wordList)

['icassp', 'improved', 'human', 'face', 'identification', 'using', 'frequency', 'domain', 'representation', 'of', 'facial', 'asymmetry', 'pattern', 'recognition', 'unsupervised', 'methods', 'for', 'the', 'classification', 'hyperspectral', 'images', 'with', 'low', 'spatial', 'resolution', 'patternrecognition', 'iscas', 'post', 'layout', 'watermarking', 'method', 'ip', 'protection', 'computers', 'mathematics', 'applications', 'some', 'tauberian', 'theorems', 'product', 'borel', 'and', 'cesàro', 'summability', 'mathematicswithapplications', 'ieee', 'geoscience', 'remote', 'sensing', 'an', 'all', 'weather', 'satellite', 'data', 'assimilation', 'retrieval', 'system', 'ieeet', 'geoscienceandremotesensing', 'journal', 'approximation', 'theory', 'convergence', 'rates', 'vector', 'cascade', 'algorithms', 'in', 'lp', 'journalofapproximationtheory', 'medical', 'informatics', 'desirability', 'expectations', 'uk', 'ms', 'register', 'views', 'people', 'medicalinformatics', 'icisc', 'fractional', 'wi

In [10]:
stop_word=['and','a','an','by','of','on','to','with','for','the','in']
wordList.extend(jour_list)
for word in stop_word:
    if word in wordList:
        wordList.remove(word)
wordList = set(wordList)

for index,doc in enumerate(docs):
    # if i%1000==0:
    #     print(i)
    new_doc = []
    for word in doc:
        if word in wordList:
            new_doc.append(word)
    docs[index] = ' '.join(new_doc)
print(docs[:100])

['icassp improved human face identification using frequency domain representation facial asymmetry icassp', 'pattern recognition unsupervised methods classification hyperspectral images low spatial resolution patternrecognition', 'iscas post layout watermarking method ip protection iscas', 'computers mathematics applications some tauberian theorems product method borel cesàro summability computers mathematicswithapplications', 'ieee geoscience remote sensing all weather satellite data assimilation retrieval system ieeet geoscienceandremotesensing', 'journal approximation theory convergence rates vector cascade algorithms lp journalofapproximationtheory', 'medical informatics desirability expectations uk ms register views people ms medicalinformatics', 'icisc fractional windows revisited improved signed digit representations efficient exponentiation icisc', 'ieice transactions public key encryptions invariant security reductions multi user setting ieicetransactions', 'isd proposed exten

In [11]:
#做TFIDF 转换
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(docs))
print(type(tfidf))
print(tfidf)
# print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 14446)	0.486639711595
  (0, 15481)	0.260822998873
  (0, 14178)	0.233794241775
  (0, 11103)	0.280196988062
  (0, 14771)	0.254554886978
  (0, 33028)	0.150007750683
  (0, 12123)	0.258141273671
  (0, 8869)	0.257628541228
  (0, 26554)	0.261626110473
  (0, 11115)	0.319106108212
  (0, 2275)	0.414000273665
  (1, 23306)	0.268108958309
  (1, 26013)	0.241620107119
  (1, 32898)	0.356742859132
  (1, 19755)	0.262255339265
  (1, 5230)	0.260244406484
  (1, 14326)	0.388341847077
  (1, 15323)	0.273336729755
  (1, 18739)	0.271599846148
  (1, 29393)	0.29568240542
  (1, 26675)	0.315353473044
  (1, 23310)	0.346124394404
  (2, 16799)	0.580475609026
  (2, 24374)	0.360936722928
  (2, 18066)	0.36883041648
  :	:
  (3053714, 6007)	0.357616480015
  (3053714, 9364)	0.339829425341
  (3053714, 24292)	0.489787771685
  (3053715, 19744)	0.166176260244
  (3053715, 6003)	0.154215653338
  (3053715, 20370)	0.203784148559
  (3053715, 1789)	0.17552160531
  (3053715, 10337)	0.2218095

In [12]:
#获取每个作者发表的文章
dfPapers.index = np.arange(len(dfPapers))
dfPapers['index'] = dfPapers.index
dfPapers['index'] = dfPapers.index.astype(str)

#get cited numbers
cited = []#cited numbers
for _ in dfPapers.index:
    cited.append('')

#print(type(dfPapers.index)) # <class 'pandas.core.indexes.numeric.Int64Index'>
#print(type(dfPapers['index'])) # <class 'pandas.core.series.Series'>

#cite = dfPapers.loc[1,['citation']]
#print(type(cite)) #<class 'pandas.core.series.Series'>
#print(list(cite))

#cite = dfPapers.loc[1,'citation']
#print(type(cite)) #<class 'str'>
#print(cite)

#for index in dfPapers['index']:
    #print(type(index)) #<class 'str'>

for index in dfPapers.index:
    cite = dfPapers.loc[index,'citation']
    if cite=='':
        continue
    for i in cite.split('\n,')[:-1]:
        if int(i)<len(dfPapers):
            cited[int(i)] = cited[int(i)] + str(index) + ':'
dfPapers['cited'] = cited
#dfPapers.drop(['year'],axis = 1,inplace = True)
print(dfPapers.loc[1,'cited'])
print(type(cited[177509]))
print('a' + cited[177509])
print(dfPapers.iloc[0:5])



<class 'str'>
a
                                              author  \
0                      Sinjini Mitra,Marios Savvides   
1  Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2        Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga   
3                            Yilmaz Erdem,Ümit Totur   
4  Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   

                                            citation index  \
0                                                        0   
1  1775098\n,1648782\n,2206608\n,2857290\n,267443...     1   
2                                                        2   
3                                           96303\n,     3   
4                                                        4   

                                     journal  \
0                                 ICASSP (2)   
1                        Pattern Recognition   
2                                  ISCAS (6)   
3  Computers & Mathematics with Applications   
4      IEEE T. Geoscience and Rem

In [13]:
dfPapers['author']+=','*22
for i in range(23):
    dfPapers['author'+str(i+1)] = dfPapers['author'].apply(lambda x:x.split(',')[i])
dfPapers['author'] = dfPapers['author'].apply(lambda x:x[:-6])
print(dfPapers[0:5])


                                              author  \
0      Sinjini Mitra,Marios Savvides,,,,,,,,,,,,,,,,   
1  Alberto Villa,Jocelyn Chanussot,Jon Atli Bened...   
2  Tingyuan Nie,Tomoo Kisaka,Masahiko Toyonaga,,,...   
3            Yilmaz Erdem,Ümit Totur,,,,,,,,,,,,,,,,   
4  Sid-Ahmed Boukabara,Kevin Garrett,Wanchun Chen...   

                                            citation index  \
0                                                        0   
1  1775098\n,1648782\n,2206608\n,2857290\n,267443...     1   
2                                                        2   
3                                           96303\n,     3   
4                                                        4   

                                     journal  \
0                                 ICASSP (2)   
1                        Pattern Recognition   
2                                  ISCAS (6)   
3  Computers & Mathematics with Applications   
4      IEEE T. Geoscience and Remote Sensing   

 

In [14]:
for i in range(23):
    t1 = dfPapers.groupby(['author'+str(i+1)])['index'].apply(lambda x:':'.join(x)).reset_index()
    print(t1)
    t1.columns = ['author','paper'+str(i+1)]
    dfTotal = pd.merge(dfTotal,t1,on=['author'],how='left').fillna('')
print(dfTotal[0:5])


                       author1  \
0             (Zhou) Bryan Bai   
1                        A Lun   
2                   A Min Tjoa   
3       A'zraa Afhzan Ab Rahim   
4             A-Imam Al-Sammak   
5              A-Nasser Ansari   
6                    A-Ning Du   
7                   A-Qun Deng   
8                   A-Ram Choi   
9                    A-Rum Jun   
10                  A-Xing Zhu   
11                 A-Youn Park   
12            A. (Zizo) Farrag   
13            A. A'Campo-Neuen   
14          A. A. (Louis) Beex   
15               A. A. A. Kock   
16             A. A. A. Nasser   
17                  A. A. Aaby   
18           A. A. Abdel Kader   
19            A. A. Abdel-Aziz   
20           A. A. Abdel-Hamid   
21           A. A. Abdel-Raouf   
22            A. A. Abdelwahab   
23              A. A. Abdurrab   
24              A. A. Abimbola   
25            A. A. Abouelsoud   
26                 A. A. Adams   
27              A. A. Adekunle   
28            

                        author3  \
0                                 
1                    A Min Tjoa   
2                 A-Chuan Hsueh   
3               A-Nasser Ansari   
4                  A-Rang Jeong   
5                    A-Reum Bae   
6                    A-Xing Zhu   
7                   A-Young Cho   
8                      A-rom So   
9            A. A. (Louis) Beex   
10             A. A. A. Darwish   
11               A. A. A. Samat   
12           A. A. Abd El-Latif   
13             A. A. Abd Elaziz   
14             A. A. Abdelhamid   
15            A. A. Abdildayeva   
16                  A. A. Abidi   
17             A. A. Abo Khadra   
18             A. A. Abouelsoud   
19               A. A. Abuzneid   
20                A. A. Agboola   
21            A. A. Albuquerque   
22              A. A. Alexndrov   
23                    A. A. Ali   
24                  A. A. Almen   
25               A. A. Altintas   
26                 A. A. Aravin   
27                 A

                         author5  \
0                                  
1                     A Min Tjoa   
2                  A. A. Adeniyi   
3          A. A. Aguilar-Laserre   
4               A. A. Ambardekar   
5                    A. A. Ameer   
6                  A. A. Andrade   
7                     A. A. Awan   
8              A. A. Azeezunnisa   
9               A. A. Dijkhuizen   
10                   A. A. Frank   
11       A. A. Holzbacher-Valero   
12            A. A. Hosseinzadeh   
13                 A. A. Joneidi   
14                 A. A. Kiselev   
15                    A. A. Kiss   
16         A. A. Letichevsky Jr.   
17          A. A. López-González   
18                  A. A. Marsan   
19                    A. A. Naik   
20                  A. A. Petrov   
21               A. A. Polezhaev   
22                   A. A. Pérez   
23                   A. A. Quivy   
24           A. A. Serafetinides   
25                    A. A. Shah   
26        A. A. Shanika Udes

                               author7  \
0                                        
1                           A-Xing Zhu   
2                          A. A. Azlan   
3                     A. A. Georgiadou   
4                           A. A. Khan   
5                          A. A. Quivy   
6                         A. A. Wilson   
7                         A. A. Zhukov   
8                           A. Abraham   
9                            A. Achour   
10                            A. Ahmad   
11                        A. Al Bi-tar   
12                        A. Amirfazli   
13                             A. Amon   
14                       A. Andreasyan   
15                           A. Arturi   
16                           A. Arvani   
17                        A. B. Bruehl   
18                        A. B. Culaba   
19                         A. B. Hmelo   
20                     A. B. M. Russel   
21                      A. B. S. Alias   
22                     A. B. Shahr

                      author9  \
0                               
1                  A Min Tjoa   
2                  A. Al-Kaff   
3                    A. Altes   
4                 A. Andreini   
5                   A. Arezzo   
6                     A. Asai   
7                  A. B. Geva   
8                A. Benvenuti   
9                  A. Berrada   
10              A. Boukabache   
11                 A. Bravaix   
12                 A. Bruschi   
13              A. C. Busacca   
14              A. C. C. Sips   
15          A. C. VanDeventer   
16                 A. Cardone   
17              A. Cody Young   
18           A. David Edwards   
19              A. Deerenberg   
20                 A. Desbrée   
21       A. E. (Rick) Seigler   
22            A. Eduardo Dias   
23           A. Eliot Shearer   
24           A. Erman Tekkaya   
25               A. F. Gazdar   
26                A. Ferreiro   
27                   A. Ferré   
28                A. Floriano   
29        

...

                  author15                                              index
0                           0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17:18...
1              A. C. Heath                                            2015966
2            A. E. Cawkell                                            1393247
3            A. E. Seigler                                             102938
4                A. Geiger                                             717489
5                A. Grover                                            1017990
6        A. Jd. O. Martins                                             528348
7             A. Kawtrakul                                            1667857
8            A. Kousaridas                                             120625
9           A. Kulshrestha                                             727760
10             A. M. Smith                                            2273185
11             A. Macerola                                      

                      author17  \
0                                
1                     A. Chang   
2            A. De Keersgieter   
3            A. J. Weatherhead   
4                      A. Jain   
5                     A. Kamal   
6                   A. Kreshuk   
7                  A. Lemagnen   
8               A. Macho Ortiz   
9                  A. Melguizo   
10                  A. Moudgal   
11                   A. Soroko   
12                   Aaron Lee   
13                Abdul Hassan   
14          Abhik Mukhopadhyay   
15       Abudouyimujiang Kader   
16         Achim J. Lilienthal   
17               Adam Fleisher   
18                   Adam Funk   
19               Adam J. Storm   
20                  Adam Lopez   
21            Adam M. Brickman   
22           Adrian M. Whatley   
23               Adrian Robert   
24                 Adriano Lai   
25               Adrien Marion   
26    Agustín Álvarez Marquina   
27                 Ajeet Singh   
28            

                    author19  \
0                              
1                 A. Brillet   
2                A. Brunetti   
3              A. Elghufaili   
4                  A. Grosso   
5                  A. Hanvey   
6                A. J. Sykes   
7               A. J. Yatish   
8             A. Paul Millar   
9                A. Yamamoto   
10            Aaron Wittmann   
11        Abdellah Fourtassi   
12         Abraham Z. Snyder   
13                Adam Padee   
14              Adam Parsons   
15             Adam Shepherd   
16         Adauto L. Mancini   
17             Aditya Akella   
18             Aditya Ramesh   
19          Aditya Rayankula   
20              Adrian Preda   
21            Akihiko Tanaka   
22          Akihiro Nitayama   
23              Alain Gateau   
24          Alan H. Fairlamb   
25                Alan Shieh   
26              Alan Sussman   
27                Alan Verlo   
28         Albert J. Vilella   
29            Albert Stadler   
...     

                 author21                                              index
0                          0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17:18...
1             A. Gregorio                                             442977
2              A. Junique                                            1540366
3                A. Mavor                                            1186064
4              A. Naumann                                            1635552
5            A. Ostrovsky                                            1904049
6      A. V. S. S. Prasad                                            2632813
7          A. W. Trafford                                            2943577
8            Aaron Mosher                                            2398672
9             Aaron Thean                                             327839
10          Aawatif Hayar                                            1418571
11          Abhinav Gupta                                            1166601

                      author23  \
0                                
1                    A. Bagchi   
2                     A. Gómez   
3                  A. Murasugi   
4               A. Purkayastha   
5                   A. Ribeiro   
6                     A. Zahid   
7               Abdenour Hadid   
8            Abraham Sebastian   
9             Abulhair Saparov   
10                  Adam Dukes   
11             Adam Strzelecki   
12            Aditi Chatterjee   
13               Adolfy Hoisie   
14                 Ahmed Rabea   
15          Akiyuki Hatakeyama   
16           Alain Phommahaxay   
17            Alain R. Thierry   
18                   Alan Fern   
19                Alberto Pace   
20           Alessandro Tocchi   
21              Alex McDougall   
22           Alexander Kanapin   
23              Alexander Senf   
24               Alexander Yip   
25            Alfonso Tarancón   
26               Ali Vahidsafa   
27                 Alice Meier   
28            

In [15]:
for i in range(23):
    t1 = dfPapers.groupby(['author'+str(i+1)])['cited'].apply(lambda x:':'.join(x)).reset_index()
    print(t1)
    t1.columns = ['author','cited'+str(i+1)]
    dfTotal = pd.merge(dfTotal,t1,on=['author'],how='left').fillna('')
print(dfTotal[0:5])


                       author1  \
0             (Zhou) Bryan Bai   
1                        A Lun   
2                   A Min Tjoa   
3       A'zraa Afhzan Ab Rahim   
4             A-Imam Al-Sammak   
5              A-Nasser Ansari   
6                    A-Ning Du   
7                   A-Qun Deng   
8                   A-Ram Choi   
9                    A-Rum Jun   
10                  A-Xing Zhu   
11                 A-Youn Park   
12            A. (Zizo) Farrag   
13            A. A'Campo-Neuen   
14          A. A. (Louis) Beex   
15               A. A. A. Kock   
16             A. A. A. Nasser   
17                  A. A. Aaby   
18           A. A. Abdel Kader   
19            A. A. Abdel-Aziz   
20           A. A. Abdel-Hamid   
21           A. A. Abdel-Raouf   
22            A. A. Abdelwahab   
23              A. A. Abdurrab   
24              A. A. Abimbola   
25            A. A. Abouelsoud   
26                 A. A. Adams   
27              A. A. Adekunle   
28            

                        author3  \
0                                 
1                    A Min Tjoa   
2                 A-Chuan Hsueh   
3               A-Nasser Ansari   
4                  A-Rang Jeong   
5                    A-Reum Bae   
6                    A-Xing Zhu   
7                   A-Young Cho   
8                      A-rom So   
9            A. A. (Louis) Beex   
10             A. A. A. Darwish   
11               A. A. A. Samat   
12           A. A. Abd El-Latif   
13             A. A. Abd Elaziz   
14             A. A. Abdelhamid   
15            A. A. Abdildayeva   
16                  A. A. Abidi   
17             A. A. Abo Khadra   
18             A. A. Abouelsoud   
19               A. A. Abuzneid   
20                A. A. Agboola   
21            A. A. Albuquerque   
22              A. A. Alexndrov   
23                    A. A. Ali   
24                  A. A. Almen   
25               A. A. Altintas   
26                 A. A. Aravin   
27                 A

                         author5  \
0                                  
1                     A Min Tjoa   
2                  A. A. Adeniyi   
3          A. A. Aguilar-Laserre   
4               A. A. Ambardekar   
5                    A. A. Ameer   
6                  A. A. Andrade   
7                     A. A. Awan   
8              A. A. Azeezunnisa   
9               A. A. Dijkhuizen   
10                   A. A. Frank   
11       A. A. Holzbacher-Valero   
12            A. A. Hosseinzadeh   
13                 A. A. Joneidi   
14                 A. A. Kiselev   
15                    A. A. Kiss   
16         A. A. Letichevsky Jr.   
17          A. A. López-González   
18                  A. A. Marsan   
19                    A. A. Naik   
20                  A. A. Petrov   
21               A. A. Polezhaev   
22                   A. A. Pérez   
23                   A. A. Quivy   
24           A. A. Serafetinides   
25                    A. A. Shah   
26        A. A. Shanika Udes

                               author7  \
0                                        
1                           A-Xing Zhu   
2                          A. A. Azlan   
3                     A. A. Georgiadou   
4                           A. A. Khan   
5                          A. A. Quivy   
6                         A. A. Wilson   
7                         A. A. Zhukov   
8                           A. Abraham   
9                            A. Achour   
10                            A. Ahmad   
11                        A. Al Bi-tar   
12                        A. Amirfazli   
13                             A. Amon   
14                       A. Andreasyan   
15                           A. Arturi   
16                           A. Arvani   
17                        A. B. Bruehl   
18                        A. B. Culaba   
19                         A. B. Hmelo   
20                     A. B. M. Russel   
21                      A. B. S. Alias   
22                     A. B. Shahr

                      author9  \
0                               
1                  A Min Tjoa   
2                  A. Al-Kaff   
3                    A. Altes   
4                 A. Andreini   
5                   A. Arezzo   
6                     A. Asai   
7                  A. B. Geva   
8                A. Benvenuti   
9                  A. Berrada   
10              A. Boukabache   
11                 A. Bravaix   
12                 A. Bruschi   
13              A. C. Busacca   
14              A. C. C. Sips   
15          A. C. VanDeventer   
16                 A. Cardone   
17              A. Cody Young   
18           A. David Edwards   
19              A. Deerenberg   
20                 A. Desbrée   
21       A. E. (Rick) Seigler   
22            A. Eduardo Dias   
23           A. Eliot Shearer   
24           A. Erman Tekkaya   
25               A. F. Gazdar   
26                A. Ferreiro   
27                   A. Ferré   
28                A. Floriano   
29        

...

                  author15                                              cited
0                           :::::495153:2656629:::197506:235072:577779:610...
1              A. C. Heath                                                   
2            A. E. Cawkell                                                   
3            A. E. Seigler                                                   
4                A. Geiger                                                   
5                A. Grover                                                   
6        A. Jd. O. Martins                                                   
7             A. Kawtrakul                                                   
8            A. Kousaridas                                                   
9           A. Kulshrestha                                                   
10             A. M. Smith           1020280:1067038:1739899:2673764:2786733:
11             A. Macerola                                      

                      author17  \
0                                
1                     A. Chang   
2            A. De Keersgieter   
3            A. J. Weatherhead   
4                      A. Jain   
5                     A. Kamal   
6                   A. Kreshuk   
7                  A. Lemagnen   
8               A. Macho Ortiz   
9                  A. Melguizo   
10                  A. Moudgal   
11                   A. Soroko   
12                   Aaron Lee   
13                Abdul Hassan   
14          Abhik Mukhopadhyay   
15       Abudouyimujiang Kader   
16         Achim J. Lilienthal   
17               Adam Fleisher   
18                   Adam Funk   
19               Adam J. Storm   
20                  Adam Lopez   
21            Adam M. Brickman   
22           Adrian M. Whatley   
23               Adrian Robert   
24                 Adriano Lai   
25               Adrien Marion   
26    Agustín Álvarez Marquina   
27                 Ajeet Singh   
28            

                    author19  \
0                              
1                 A. Brillet   
2                A. Brunetti   
3              A. Elghufaili   
4                  A. Grosso   
5                  A. Hanvey   
6                A. J. Sykes   
7               A. J. Yatish   
8             A. Paul Millar   
9                A. Yamamoto   
10            Aaron Wittmann   
11        Abdellah Fourtassi   
12         Abraham Z. Snyder   
13                Adam Padee   
14              Adam Parsons   
15             Adam Shepherd   
16         Adauto L. Mancini   
17             Aditya Akella   
18             Aditya Ramesh   
19          Aditya Rayankula   
20              Adrian Preda   
21            Akihiko Tanaka   
22          Akihiro Nitayama   
23              Alain Gateau   
24          Alan H. Fairlamb   
25                Alan Shieh   
26              Alan Sussman   
27                Alan Verlo   
28         Albert J. Vilella   
29            Albert Stadler   
...     

                 author21                                              cited
0                          :::::495153:2656629:::197506:235072:577779:610...
1             A. Gregorio  145652:331652:395176:709810:826661:1582283:160...
2              A. Junique                                                   
3                A. Mavor                                                   
4              A. Naumann                                                   
5            A. Ostrovsky                                                   
6      A. V. S. S. Prasad                                                   
7          A. W. Trafford                                                   
8            Aaron Mosher                                                   
9             Aaron Thean                                                   
10          Aawatif Hayar                                                   
11          Abhinav Gupta                                                   

                      author23  \
0                                
1                    A. Bagchi   
2                     A. Gómez   
3                  A. Murasugi   
4               A. Purkayastha   
5                   A. Ribeiro   
6                     A. Zahid   
7               Abdenour Hadid   
8            Abraham Sebastian   
9             Abulhair Saparov   
10                  Adam Dukes   
11             Adam Strzelecki   
12            Aditi Chatterjee   
13               Adolfy Hoisie   
14                 Ahmed Rabea   
15          Akiyuki Hatakeyama   
16           Alain Phommahaxay   
17            Alain R. Thierry   
18                   Alan Fern   
19                Alberto Pace   
20           Alessandro Tocchi   
21              Alex McDougall   
22           Alexander Kanapin   
23              Alexander Senf   
24               Alexander Yip   
25            Alfonso Tarancón   
26               Ali Vahidsafa   
27                 Alice Meier   
28            

In [16]:
# print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号
print(tfidf[1][0])
tfidf_weight_array = np.zeros([len(dfTotal),tfidf.shape[1]])
# print(dfTotal.shape) # there is 11367 authors
# print(dfPapers.shape) # there is 3053717 papers
# print(tfidf_weight_array.shape) # (11367, 34821)

for i in range(23):
#for i in range(2):
    col_name = 'paper'+str(i+1)
    #for j in [0,1]:
    for j in dfTotal.index:
        paperlist = dfTotal.loc[j,col_name].split(':')
        #print(paperlist)
        for paper_index in paperlist:
            if paper_index =='':
                continue
            tfidf_weight = tfidf[int(paper_index)][0]
            # print(type(tfidf_weight)) # <class 'scipy.sparse.csr.csr_matrix'>
            #print(tfidf_weight)
            for words in tfidf_weight.indices: # csr_matrix.indices表示各个数据在某行的下标，即第几列，下标从0开始
                #print(words)
                if j<7:
                    # csr_matrix.getcol(i),获得矩阵的第i列，返回一个列向量
                    tfidf_weight_array[j,words] = tfidf_weight.getcol(words).data[0]*(1-0.1*i)
                if j>=7:
                    tfidf_weight_array[j,words] = tfidf_weight.getcol(words).data[0]*0.3
print(tfidf_weight_array[0:5,:])

  (0, 23306)	0.268108958309
  (0, 26013)	0.241620107119
  (0, 32898)	0.356742859132
  (0, 19755)	0.262255339265
  (0, 5230)	0.260244406484
  (0, 14326)	0.388341847077
  (0, 15323)	0.273336729755
  (0, 18739)	0.271599846148
  (0, 29393)	0.29568240542
  (0, 26675)	0.315353473044
  (0, 23310)	0.346124394404
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [17]:
print(type(tfidf_weight_array))
print(tfidf_weight_array[0,:])

for i in range(7):
    col_name = 'cited'+str(i+1)
    for j in dfTotal.index:
        paperlist = dfTotal.loc[j,col_name].split(':')
        for paper_index in paperlist:
            if paper_index =='':
                continue
            tfidf_weight = tfidf[int(paper_index)][0]
            for words in tfidf_weight.indices:
                #print(words)
                tfidf_weight_array[j,words] = tfidf_weight.getcol(words).data[0]*(1-i*0.1)*0.1
print(tfidf_weight_array[0:5,:])

<class 'numpy.ndarray'>
[ 0.  0.  0. ...,  0.  0.  0.]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [18]:
'''
用keras 的普通的DNN训练
共700+种class
label里出现的class 按出现顺序分别给 0.95 0.85 0.75 的分数，其他给0.1
'''

labels = pd.read_csv('/Users/yuansha/Desktop/portrait/data of task2/labels.txt',header = None)
y_total = 0.1 * np.ones([len(dfTotal),len(labels)],dtype='float64')
print(y_total.shape)

labels['idx'] = labels.index
labels.index = labels[0]
labels.drop(0,axis = 1,inplace = True)

#print(dfTrain.iloc[1])
#print(dfTrain.loc[1,'interest1'])

for idx in dfTrain.index:
    for j in range(3):
        y_total[idx,labels.loc[dfTrain.loc[idx,'Interest'+str(j+1)],'idx']] = 0.95-0.1*j
print(y_total[0:5])


(11367, 789)
[[ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]
 [ 0.1  0.1  0.1 ...,  0.1  0.1  0.1]]


In [19]:
idtolabels = labels.copy()
print(type(idtolabels))
idtolabels['name'] = idtolabels.index
idtolabels.index = idtolabels.idx
idtolabels.drop('idx',inplace = True,axis = 1)
print(idtolabels.iloc[0:5])

<class 'pandas.core.frame.DataFrame'>
                        name
idx                         
0    abstract interpretation
1             access control
2             access network
3            active learning
4       activity recognition


In [ ]:
class Task2_Model():

        def __init__(self,output_dim):
            '''
            model1: tfidf
            '''
            self.model = Sequential()
            self.model.add(Dense(1000,activation='relu',input_dim=tfidf_weight_array.shape[1]))
            # print(tfidf.shape) # (3053717, 34821) 行数为文档编号，列数为单词编号
            self.model.add(Dense(output_dim,activation='sigmoid'))

            self.model.compile(loss='categorical_crossentropy',
                                optimizer='adadelta',
                                metrics=['acc'])

        def train_model(self,x_train,y_train,sp,epochs):
            if sp < 1.0:
                self.model.fit(x_train[0:int(len(x_train)*sp), :],y_train[0:int(len(x_train)*sp),:],
                                batch_size=30,
                                epochs=epochs,
                                validation_data=(x_train[int(len(x_train)*sp):,:],y_train[int(len(y_train)*sp):,:]))

            if sp == 1.0:
                self.model.fit(x_train,y_train,
                            batch_size=30,
                            epochs=epochs,
                            validation_data=(x_train,y_train))
            history = self.model.fit.history

        def save_model(self,path):
            self.model.save(path + '_1_lda.h5')

        def load_model(slef, path):
            self.model.load_weights(path +'_1_lda.h5')

        def predict(self,fea1):
            submission = list()
            preds = self.model.predict(fea1)
            for idx in range(len(fea1)):
                a = np.argsort(preds[idx])[-5:] # np.argsort升序排列并返回下标
                a_label = idtolabels.ix[a].values
                # print(a_label)
                submission.append(a_label)
            return submission


        def getScore(self,X,y,idtolabel):
            preds = self.model.predict(X)
            score = 0
            for idx in range(len(X)):
                a = np.argsort(preds[idx])[-3:]
                a_label = idtolabel.ix[a].values
                # print(a_label)
                for j in range(3):
                    for k in range(3):
                        if a_label[j]==y[idx,k]:
                            score+=1
                            break
            return score/len(X)/3
        def summary(self):
            print(self.model.summary())


In [ ]:
model = Task2_Model(len(labels))
hist = model.train_model(tfidf_weight_array[:6000],y_total[:6000],1,13)
    # print(model.getScore(tfidf_weight_array[4800:6000],dfTrain.ix[4800:6000,1:].values,idtolabels))
submission = model.predict(tfidf_weight_array[6000:])


Train on 6000 samples, validate on 6000 samples
Epoch 1/13
6000/6000 [==============================] - 142s 24ms/step - loss: 541.1954 - acc: 0.0447 - val_loss: 541.1297 - val_acc: 0.0460
Epoch 2/13
6000/6000 [==============================] - 132s 22ms/step - loss: 541.0887 - acc: 0.0602 - val_loss: 541.0165 - val_acc: 0.1037
Epoch 3/13
6000/6000 [==============================] - 132s 22ms/step - loss: 540.9739 - acc: 0.1405 - val_loss: 540.8957 - val_acc: 0.1712
Epoch 4/13
6000/6000 [==============================] - 132s 22ms/step - loss: 540.8468 - acc: 0.1938 - val_loss: 540.7431 - val_acc: 0.1945
Epoch 5/13
6000/6000 [==============================] - 136s 23ms/step - loss: 540.6921 - acc: 0.2430 - val_loss: 540.5517 - val_acc: 0.3028
Epoch 6/13
6000/6000 [==============================] - 133s 22ms/step - loss: 540.5174 - acc: 0.2962 - val_loss: 540.3611 - val_acc: 0.3525
Epoch 7/13
6000/6000 [==============================] - 138s 23ms/step - loss: 540.3341 - acc: 0.3597 - va

/Users/yuansha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [22]:
# the function plot loss and acc
def training_vis(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    acc = hist.history['acc']
    val_acc = hist.history['val_acc']

    # make a figure
    fig = plt.figure(figsize=(8,4))
    # subplot loss
    ax1 = fig.add_subplot(121)
    ax1.plot(loss,label='train_loss')
    ax1.plot(val_loss,label='val_loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    ax1.legend()
    # subplot acc
    ax2 = fig.add_subplot(122)
    ax2.plot(acc,label='train_acc')
    ax2.plot(val_acc,label='val_acc')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Accuracy  on Training and Validation Data')
    ax2.legend()
    plt.tight_layout()

In [23]:
training_vis(hist)

AttributeError: 'NoneType' object has no attribute 'history'

In [24]:
print(hist.history)

AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
with open('validation2.csv','w') as f:
    f.write('<task2>\nname')
    for i in range(5):
        f.write('\tinterest'+str(i+1))
    f.write('\n')
    for i in range(len(submission)):
        f.write('%s'%(dfValidation.ix[i,0]))
        for j in range(5):
            f.write('\t%s'%(submission[i][j][0]))
        f.write('\n')
    f.write('</task2>')